In [1]:
import os
import time
import math
import heapq
from datetime import datetime
from IPython.display import clear_output

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

In [2]:
# change working directory
os.chdir(os.getcwd())

collated_df = pd.read_csv('collated_data.csv')
df_gld = pd.read_csv('./data/GLD (Gold Shares 01-01-2008 to 03-07-2022).csv')
df_spy = pd.read_csv('./data/SPY (S&P500 ETF 01-01-2008 to 03-07-2022)).csv')
df_tlt = pd.read_csv('./data/TLT (US Treasury Bonds ETF 01-01-2008 to 03-07-2022).csv')
df_sgd = pd.read_csv('./data/SGD=X (01-01-2008 to 03-07-2022).csv')
df_oil = pd.read_csv('./data/USO (US Oil Index 01-01-2008 to 03-07-2022).csv')

collated_dict = {'Unemployment Rate': 'UNRATE', 'GDP': 'GDP', 'INTEREST RATES': 'FEDFUNDS',
                 'CPI DATA': 'CPALTT01USM657N', 'TOTAL FED ASSET': 'D2WATAL'}
df_dict = {'GLD': df_gld, 'SPY': df_spy, 'TLT': df_tlt, 'SGD': df_sgd, 'OIL': df_oil}

#### Background function for visualization of correlation, daily closing & cul. returns

In [3]:
def setup_index_data(df_spy, df_gld, df_tlt, df_oil):
    # Create a new dataframe having only closing prices of each
    index_data = pd.DataFrame({'SPY':df_spy['Close'], 'GLD':df_gld['Close'], 
                               'TLT':df_tlt['Close'], 'OIL':df_oil['Close']})
    return index_data

def priceCorrPrint(df):
    
    correlation = df.corr("pearson")
    plt.subplots(figsize=(5,5))
    sns.heatmap(correlation, annot=True, fmt='.2%')

    sns.histplot(df['SPY'],color='green')
    
    return

def plotDailyClosing(df):
    
    plt.figure(figsize=(12.2,4.5))

    for c in df.columns.values:
        plt.plot(df[c], label=c, lw=0.5, alpha=1)

    plt.title('Daily Closing Price')
    plt.xlabel('No. of days (01-01-2008 to 03-07-2022)')
    plt.xticks(rotation=10)
    plt.ylabel('Price in USD')
    plt.legend(df.columns.values, loc='upper right')
    plt.show()
    
    return 

def culSimpleReturn(df):
    simpleReturn = df.pct_change(1)
    culSimpleReturn = (simpleReturn+1).cumprod()

    plt.figure(figsize=(12.2,4.5))

    for c in culSimpleReturn.columns.values:
        plt.plot(culSimpleReturn[c], label=c, lw=0.5, alpha=1)

    plt.title('Cumulative Simple Returns')
    plt.xlabel('No. of days (01-01-2008 to 03-07-2022)')
    plt.xticks(rotation=10)
    plt.ylabel('USD growth / USD')
    plt.legend(df.columns.values, loc='upper right')
    plt.show()
    
    return 

#### Driver functions for visualization of correlation, daily closing & cul. returns

In [4]:
def driver_viz(df_spy, df_gld, df_tlt, df_oil):
    
    index_df = setup_index_data(df_spy, df_gld, df_tlt, df_oil)
    priceCorrPrint(index_df)
    print('')
    print('')
    time.sleep(1)
    plotDailyClosing(index_df)
    time.sleep(1)
    culSimpleReturn(index_df)
    
    return

#### Background functions for visualization of trading signals

In [5]:
def setup_moving_avg_list(df, shortTermWindow=13, longTermwindow=55):
    
    #use fibonacci no. for STMA and LTMA: 13, 55 (default values)
    shortTermWindow = shortTermWindow
    longTermWindow = longTermwindow
    
    #setting up the new dataframe
    STMA = pd.DataFrame() 
    STMA['Close']= df['Close'].rolling(window = shortTermWindow).mean()

    LTMA = pd.DataFrame() 
    LTMA['Close']= df['Close'].rolling(window = longTermWindow).mean()
    
    moving_avg = pd.DataFrame()
    moving_avg['PRICE'] = df['Close']
    moving_avg['STMA'] = STMA['Close']
    moving_avg['LTMA'] = LTMA['Close']
    moving_avg.tail()
    
    return moving_avg, STMA, LTMA

def signals(df):
    
    sigBuy = []
    sigSell = []
    crossOver = -1 # checker: 1 means STMA crossing above LTMA (buy), 0 means LTMA crossing above STMA (sell)

    for i in range(len(df)):
        if df['STMA'][i] < df['LTMA'][i]:
            if crossOver != 0:
                sigBuy.append(np.nan)
                sigSell.append(df['PRICE'][i])
                crossOver = 0
            else: 
                sigBuy.append(np.nan)
                sigSell.append(np.nan)

        elif df['STMA'][i] > df['LTMA'][i]:
            if crossOver != 1:
                sigBuy.append(df['PRICE'][i])
                sigSell.append(np.nan)
                crossOver = 1
            else:
                sigBuy.append(np.nan)
                sigSell.append(np.nan)
            
        else: 
            sigBuy.append(np.nan)
            sigSell.append(np.nan)

    return (sigBuy, sigSell)

def consolidate_signals(df):
    
    buy_sell = signals(df)
    df['Buy Price'] = buy_sell[0]
    df['Sell Price'] = buy_sell[1]

    return df

def plot_signals(instrument_df, STMA, LTMA, moving_avg, instrument):
    
    num_days = (pd.to_datetime(instrument_df.iloc[-1,0]) - pd.to_datetime(instrument_df.iloc[0,0])).days
    
    plt.figure(figsize=(12.2,4.5))

    plt.plot(instrument_df['Close'], label=instrument, lw=1, alpha=1)
    plt.plot(STMA['Close'], label='STMA', lw=1, alpha=1)
    plt.plot(LTMA['Close'], label='LTMA', lw=1, alpha=1)

    plt.scatter(moving_avg.index, moving_avg['Buy Price'], label = 'Buy', marker = "^", color="green")

    plt.scatter(moving_avg.index, moving_avg['Sell Price'], label = 'Sell', marker = "v", color="red")

    plt.title(f'{instrument} Close 1000 Days Price History- Buy & Sell Signals')
    plt.xlabel(f'{num_days} Days')
    plt.ylabel('Adj. Close Price USD ($)')
    plt.legend(loc='upper left')
    plt.show()
    
    return

#### Driver functions for visualization of trading signals

In [6]:
def driver_viz_signals(instrument, df_dict):
    
    df = df_dict[instrument]
    moving_avg, STMA, LTMA = setup_moving_avg_list(df)
    moving_avg = consolidate_signals(moving_avg)
    plot_signals(df, STMA, LTMA, moving_avg, instrument)
    
    return

#### Background functions for min-max

In [7]:
# function to return the row(s) with the max or min of the interesting data point
# call with series in data frame and quest (min or max)
# max() and min() python methods are O(n)
def dfRow(df, series, quest):
    
    val = None
    start = time.time()
    
    if quest == 'max': 
        # find max value in series
        val = series.max()
    if quest == 'min':
        # find min value in series
        val = series.min()
    # if quest is invalid
    if quest != 'max' and quest != 'min':
        print('Second argument is neither max or min. Try again.')
        return
    
    end = time.time()
    
    # prepare df filter/sieve
    sieve = series.isin([val])
    
    time_taken = round(end - start, 10)
    print('The amount of time taken is: ', time_taken)
    
    # return df row(s) with maxVal
    return df[sieve]

# function to return the row(s) with the max or min of the interesting data point
# call with series in data frame and quest (min or max)
def dfRowAlt(df, series, quest):
    
    # filer out NaN values
    filteredSeries = series.dropna()
    
    # convert to pyton list for heapq lib
    lst = filteredSeries.tolist()
    
    # turn python list into heap
    heapq.heapify(lst)
    
    val = None
    startAlt = time.time()
    
    if quest == 'max': 
        # find max value
        val = heapq.nlargest(1, lst)[0]
    if quest == 'min':
        # find min value
        val = heapq.nsmallest(1, lst)[0]
    # if quest is invalid
    if quest != 'max' and quest != 'min':
        print('Second argument is neither max or min. Try again.')
        return
    
    endAlt = time.time()
    
    # prepare df filter/sieve
    sieve = series.isin([val])
    
    time_taken = round(endAlt - startAlt, 10)
    print('The amount of time taken is: ', time_taken)
    
    # return df row(s) with maxVal
    return df[sieve]

#### Driver functions for min-max

In [8]:
def driver_min_max(parameter, collated_dict):
    parameter1 = collated_dict[parameter]
    df = collated_df.loc[:, ['Year', 'Month', 'Day', parameter1]]
    print('Using python\'s max() & min() methods; time complexity = O(n) . . .')
    print('')
    print('Finding min . . .')
    print('')
    res = dfRow(collated_df, collated_df[parameter1], 'min')
    print(res.to_markdown())
    print('')
    time.sleep(1)
    print('Finding max . . .')
    print('')
    res1 = dfRow(collated_df, collated_df[parameter1], 'max')
    print(res1.to_markdown())
    print('')
    print('')
    print('')
    time.sleep(1)
    print('Using an alternative method . . .')
    print('')
    print('Finding min . . .')
    print('')
    res2 = dfRowAlt(collated_df, collated_df[parameter1], 'min')
    print(res2.to_markdown())
    print('')
    print('')
    time.sleep(1)
    print('Finding max . . .')
    print('')
    res3 = dfRowAlt(collated_df, collated_df[parameter1], 'max')
    print(res3.to_markdown())
    return
    

#### Background functions for finding max profit

In [9]:
def setup_buysell_df(instrument_df, startDate='2020-06-20', cutoffDate='2020-07-01'):
    
    new_df = pd.DataFrame({'DATE': instrument_df['Date'], 'PRICE': instrument_df['Close']})
    new_df['DATE'] = pd.to_datetime(new_df['DATE'])
    #new_df.tail()

    mask = (new_df['DATE'] > startDate) & (new_df['DATE'] <= cutoffDate)
    df = new_df.loc[mask]
    dailyPrice = df['PRICE'].to_numpy()
    
    return dailyPrice

### assuming 1 trade only, what is the maxprofit

### brute force method

def longOnly1(dailyPrice):
    
    # compare each element find max diff O(N^2) for time, O(1) for space
    
    n = len(dailyPrice)
    maxProfit = 0
    
    for i in range(n):
        for j in range(i+1,n):
            currProfit = dailyPrice[j] - dailyPrice[i]
            if(currProfit > maxProfit):
                maxProfit = currProfit
                
    return maxProfit

### recursive method

def longOnly2(dailyPrice):
    # using recursion O(N) for time, O(1) for space
    n = len(dailyPrice)
    return recur(n, dailyPrice)

def recur(n, dailyPrice):
    
    if n == 0:
        return 0
    else: 
        currProfit = recur(n-1, dailyPrice)
        for i in range(n):
            currProfit = max(currProfit, dailyPrice[n-1] - dailyPrice[i])
        
    return currProfit

# using min track and max tracker, traverse for the currProfit and update max tracker if exceeds O(N) for time, O(1) for space

def longOnly3(dailyPrice):
    
    n = len(dailyPrice)
    currMin = 9999
    currMax = 0
    
    for i in range(n):
        currProfit = dailyPrice[i] - currMin
        currMax = max(currProfit, currMax)
        currMin = min(currMin, dailyPrice[i])
    
    return currMax

# reverse the order of the list and apply the same longOnly algorithm

def shortOnly1(dailyPrice):
    reverseDailyPrice = dailyPrice[::-1]
    return longOnly3(reverseDailyPrice)

# assuming can take 2 long only trades

def long2Trades(dailyPrice):
    
    n = len(dailyPrice)
    entry1 = entry2 = float('inf')
    profit1 = profit2 = 0
    
    for i in range(n):
        entry1 = min(entry1, dailyPrice[i])
        profit1 = max(profit1, dailyPrice[i] - entry1)
        entry2 = min(entry2, dailyPrice[i] - profit1)
        profit2 = max(profit2, dailyPrice[i] - profit2)
        
    return profit2

# unlimited long trades 

def longTrades(dailyPrice):
    n = len(dailyPrice)
    currProfit = 0
    noOfTrades = 0
    for i in range(n-1):
        if (dailyPrice[i+1] - dailyPrice[i]>0):
            currProfit += dailyPrice[i+1] - dailyPrice[i]
            noOfTrades += 1
    return currProfit, noOfTrades

#### Driver functions for finding max profit

In [10]:
def driver_finding_max_profit(instrument, df_dict, startDate, cutoffDate):
    
    instrument_df = df_dict[instrument]
    dailyPrice_df = setup_buysell_df(instrument_df, startDate, cutoffDate)
    
    print('Find max profit assuming only 1 buy/sell trade only')
    print('')
    
    print('Running naive algorithm . . .')
    maxProfit1 = longOnly1(dailyPrice_df)
    print('Max profit is:', maxProfit1)
    print('')
    print('Running recursive algorithm . . .')
    maxProfit2 = longOnly2(dailyPrice_df)
    print('Max profit is:', maxProfit2)
    print('')
    print('Running min-max tracker algorithm . . .')
    maxProfit3 = longOnly3(dailyPrice_df)
    print('Max profit is:', maxProfit3)
    print('')
    
    print('#'*30)
    print('')
    time.sleep(1)
    
    print('Finding best margins for short trading . . .')
    maxMargin = longOnly3(dailyPrice_df)
    print('Best margin is:', maxMargin)
    print('')
    
    print('#'*30)
    print('')
    time.sleep(1)
    
    print('Find max profit assuming only TWO buy/sell trades only')
    maxProfit4 = long2Trades(dailyPrice_df)
    print('Max profit is:', maxProfit4)
    print('')
    
    print('#'*30)
    print('')
    time.sleep(1)
    
    print('Find max profit assuming UNLIMITED buy/sell trades')
    maxProfit5 = longTrades(dailyPrice_df)
    print('Max profit is:', maxProfit5)
    
    return

#### Background functions for detailed max profit

In [11]:
def date_adjustment(df):
    
    df['Date'] = pd.to_datetime(df['Date'])
    cols = ['Year', 'Month', 'Day']
    cols.extend(df.columns.tolist())
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df.sort_values(by=['Date'], inplace=True)
    df = df.reindex(columns=cols)
    df.drop(['Date'], axis=1, inplace=True)
    
    return df

def new_df(df):
    
    df_adj = pd.DataFrame()
    df_adj['Date'] = pd.to_datetime(df['Date'].values)
    df_adj['Price'] = df['Adj Close'].values
    #df_adj['SMA30'] = df_adj[instrument].rolling(window=30).mean()
    #df_adj['SMA100'] = df_adj[instrument].rolling(window=100).mean()
    
    return df_adj

def buy_sell_kadane(signal):
    
    if len(signal) < 2:
        return None
    
    start = time.time()
    
    curr_buy = signal.iloc[0,1]
    glob_sell = signal.iloc[1,1]
    glob_profit = glob_sell - curr_buy
    curr_profit = float('-inf')
    
    curr_buy_time = signal.iloc[0,0]
    glob_sell_time = signal.iloc[1,0]
    res = []
    
    for i in range(1, len(signal)):
        curr_profit = signal.iloc[i,1] - curr_buy
        
        if curr_profit > glob_profit:
            glob_profit = curr_profit
            glob_sell = signal.iloc[i,1]
            glob_sell_time = signal.iloc[i,0]
        
        if curr_buy > signal.iloc[i,1]:
            curr_buy = signal.iloc[i,1]
            curr_buy_time = signal.iloc[i,0]
    
    res.append([curr_buy_time, curr_buy, glob_sell_time, glob_sell, glob_sell - curr_buy])
    res = pd.DataFrame(res, columns = ['Buy Date', 'Buy Amount', 'Sell Date',
                                       'Sell Amount', 'Profit'])
    
    end = time.time()
    time_taken = round(end - start, 3)
    
    return res, time_taken

def buy_sell_brute(signal):
    
    if len(signal) < 2:
        return None
    
    start = time.time()
    res = []
    
    for i in range(0, len(signal)):
        curr_buy = signal.iloc[i,1]
        curr_buy_time = signal.iloc[i,0]
        curr_max_profit = float('-inf')
        
        for j in range(i, len(signal)):
            curr_sell = signal.iloc[j,1]
            curr_profit = curr_sell - curr_buy
            
            if curr_profit > curr_max_profit:
                curr_max_profit = curr_profit
                curr_max_sell = signal.iloc[j,1]
                curr_sell_time = signal.iloc[j,0]
            
            if curr_buy > signal.iloc[j,1]:
                curr_buy = signal.iloc[j,1]
                curr_buy_time = signal.iloc[j,0]
        
        res.append([curr_buy_time, curr_buy, curr_sell_time, curr_max_sell, curr_max_sell - curr_buy])
    
    res = pd.DataFrame(res, columns = ['Buy Date', 'Buy Amount', 'Sell Date',
                                       'Sell Amount', 'Profit'])
    final_res = res[res['Profit']==res['Profit'].max()].drop_duplicates()
    
    end = time.time()
    time_taken = round(end - start, 3)
    
    return final_res, time_taken

def driver_run_kadane(signal, compare=False):
    
    print('Running optimized algorithm . . .')
    kadane_res, kadane_time = buy_sell_kadane(signal)
    print('Buy Date:', kadane_res.iloc[0,0], 'Buy Price:', kadane_res.iloc[0,1])
    print('Sell Date:', kadane_res.iloc[0,2], 'Sell Price:', kadane_res.iloc[0,3])
    print('Total profit:', kadane_res.iloc[0,-1])
    print('Time taken:', round(kadane_time,3))
    
    if compare is True:
        return kadane_time
    else:
        return

def driver_run_brute(signal, compare=False):
    
    print('Running brute force algorithm . . .')
    brute_res, brute_time = buy_sell_brute(signal)
    print('Buy Date:', brute_res.iloc[0,0], 'Buy Price:', brute_res.iloc[0,1])
    print('Sell Date:', brute_res.iloc[0,2], 'Sell Price:', brute_res.iloc[0,3])
    print('Total profit:', brute_res.iloc[0,-1])
    print('Time taken:', round(brute_time,3))
    
    if compare is True:
        return brute_time
    else:
        return

def driver_compare_time(signal):
    
    kadane_time = driver_run_kadane(signal, compare=True)
    
    print('')
    time.sleep(1)
    
    brute_time = driver_run_brute(signal, compare=True)
    
    print('')
    
    if kadane_time < brute_time:
        print('Kadane algorithm runs faster.')
        print('Time difference:', round(brute_time-kadane_time,3))
    elif brute_time > kadane_time:
        print('Brute force algorthim runs faster.')
        print('Time difference:', round(kadane_time-brute_time,3))
    else:
        print('Both takes the same amount of time.')
    
    return

#### Driver functions for detailed max profit

In [12]:
def driver_detailed_profit(instrument, df_dict):
    
    df = df_dict[instrument]
    #df = date_adjustment(df)
    df = new_df(df)
    driver_compare_time(df)
    
    return

#### Menu functions

In [13]:
def instrument_menu():
    
    print('Pick instrument:')
    print('A: GLD')
    print('B: SPY')
    print('C: TLT')
    print('D: SGD')
    print('E: OIL')
    choice = input('SELECT CHOICE:  ')
    print('')
    
    return choice.upper()

def instrument_menu_choice_conversion(choice):
    
    choice_dict = {'A': 'GLD', 'B': 'SPY', 'C': 'TLT', 'D': 'SGD', 'E': 'OIL'}
    instrument = choice_dict[choice]
    
    return instrument

def parameter_menu():
    
    print('Pick parameter:')
    print('A: Unemployment Rate')
    print('B: GDP')
    print('C: INTEREST RATES')
    print('D: CPI DATA')
    print('E: TOTAL FED ASSET')
    choice = input('SELECT CHOICE:  ')
    print('')

    return choice.upper()

def parameter_menu_choice_conversion(choice):
    
    choice_dict = {'A': 'Unemployment Rate', 'B': 'GDP', 'C': 'INTEREST RATES',
                   'D': 'CPI DATA', 'E': 'TOTAL FED ASSET'}
    parameter = choice_dict[choice]
    
    return parameter
    
def input_dates():
    
    print('Pick starting and ending dates in YYYY-MM-DD format')
    print('Dates should range from 2008-01-02 to 2022-07-01')
    startDate = input('START DATE:  ')
    endDate = input('END DATE:  ')
    print('')
    
    if date_checker(startDate) is False or date_checker(endDate) is False:
        print('WARNING: Please key in valid dates within the stated date range.')
        startDate, endDate = input_dates()
        
    return startDate, endDate

def date_checker(date_input):
    
    date_input = datetime.fromisoformat(date_input).date()
    lower_bound = datetime.fromisoformat('2008-01-02').date()
    upper_bound = datetime.fromisoformat('2022-07-01').date()
    
    if date_input < lower_bound or date_input > upper_bound:
        return False
    else:
        return True

def main_menu():
    
    print('')
    print('Pick feature:')
    print('A: Find max profits for long & short trading variations')
    print('B: Find min and max of instrument')
    print('C: Deep dive of profits for 1x long trade')
    print('D: Visualizations of instruments (correlation, cumulative simple returns)')
    print('E: Visualizaiton of buy-sell signals of instrument')
    print('Z: Quit application')
    choice = input('SELECT CHOICE:  ')
    print('')
    
    return choice.upper()

### Run application

In [ ]:
main_selection = 'X'
while main_selection != 'Z':
    
    clear_output(wait=True)
    time.sleep(1)
    
    if main_selection == 'A':
        instrument_choice = instrument_menu()
        instrument = instrument_menu_choice_conversion(instrument_choice)
        startDate, endDate = input_dates()
        clear_output(wait=True)
        time.sleep(1)
        driver_finding_max_profit(instrument, df_dict, startDate, endDate)
    elif main_selection == 'B':
        parameter_choice = parameter_menu()
        parameter = parameter_menu_choice_conversion(parameter_choice)
        clear_output(wait=True)
        time.sleep(1)
        driver_min_max(parameter, collated_dict)
    elif main_selection == 'C':
        instrument_choice = instrument_menu()
        instrument = instrument_menu_choice_conversion(instrument_choice)
        clear_output(wait=True)
        time.sleep(1)
        driver_detailed_profit(instrument, df_dict)
    elif main_selection == 'D':
        driver_viz(df_spy, df_gld, df_tlt, df_oil)
    elif main_selection == 'E':
        instrument_choice = instrument_menu()
        instrument = instrument_menu_choice_conversion(instrument_choice)
        clear_output(wait=True)
        time.sleep(1)
        driver_viz_signals(instrument, df_dict)
    else: pass
    
    if main_selection != 'X':
        print('')
        input("Press Enter to continue...")
        clear_output(wait=True)
        time.sleep(1)
    
    main_selection = main_menu()


Pick feature:
A: Find max profits for long & short trading variations
B: Find min and max of instrument
C: Deep dive of profits for 1x long trade
D: Visualizations of instruments (correlation, cumulative simple returns)
E: Visualizaiton of buy-sell signals of instrument
Z: Quit application
